In [3]:
import requests
import pandas as pd


In [4]:
## Fetch an OAUTH Token. It is only valid for 2 hours and will need to be re-requested when it expires.

# The following code curtesy of James Briggs: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('fOIoNDAQnOFBiQ5RM9CEuA', 'Rb9whMn_xOwNVra9usbbEPYkVtMqvw')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'flatest_iron',
        'password': 'FlatironPassword19!'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [6]:
# This is just a test to see how this works in general

# Get the top 25 posts of all time from r/python
#TODO: It might be worth it to change which posts are pulled: maybe hot posts are more indicative, but for now we're doing this
#   Also, there is a way to get more data from each subreddit
response = requests.get("https://oauth.reddit.com/r/python/top/?t=all", headers=headers)

# python_df = pd.DataFrame()

# #We can add to this if need be; there is a lot of data that isn't being captured here
# for post in response.json()['data']['children']:
#     python_df = python_df.append({
#         'subreddit' : post['data']['subreddit'],
#         'title' : post['data']['title'],
#         'text' : post['data']['selftext'],
#         'upvote_ratio' : post['data']['upvote_ratio'],
#         'url_overriden_by_dest' : post['data']['url_overridden_by_dest'] #I think this is the image url
#     }, ignore_index = True)

# python_df.head()

In [13]:
response.json()['data']['children'][0]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'libsofreddit',
  'selftext': '',
  'author_fullname': 't2_53263itc',
  'saved': False,
  'mod_reason_title': None,
  'gilded': 0,
  'clicked': False,
  'title': 'Defending a child rapist has real life consequences for those close to the victim.',
  'link_flair_richtext': [],
  'subreddit_name_prefixed': 'r/libsofreddit',
  'hidden': False,
  'pwls': 7,
  'link_flair_css_class': '',
  'downs': 0,
  'thumbnail_height': 136,
  'top_awarded_type': None,
  'hide_score': False,
  'name': 't3_r31549',
  'quarantine': False,
  'link_flair_text_color': 'light',
  'upvote_ratio': 0.97,
  'author_flair_background_color': '#014980',
  'ups': 1222,
  'total_awards_received': 1,
  'media_embed': {},
  'thumbnail_width': 140,
  'author_flair_template_id': '3692a2d6-5228-11ec-9492-0a13995943f9',
  'is_original_content': False,
  'user_reports': [],
  'secure_media': None,
  'is_reddit_media_domain': True,
  'is_meta': False,
  'category

In [11]:
# Create a list of a subreddits that we want to fetch the top posts for

# There might be a better way to do this, but for right now, I'm just gonna do it manually
# My reasoning for this is that I can grab some less popular reddits that some of the listings
# Right now I have some blatantly right, blatantly left, and apolitical subreddits

# Hopefully I don't introduce any implicit bias into my model lol
subreddits = [
    #apolitical (I think)
    'BlackPeopleTwitter',
    'WhitePeopleTwitter',
    'politics',
    'news',
    'Jokes',
    'funny',
    'gaming',
    'technology',
    'dataisbeautiful',
    'MurderedByWords',
    'mildlyinteresting',
    'wallstreetbets',
    'shitposting',
    'pcmasterrace',
    'teenagers',
    'PublicFreakout',
    'memes',
    'meirl',
    'AskReddit',
    'GamingCircleJerk',
    'dndmemes',

    #left (I think)
    'antiwork',
    'SelfAwarewolves',
    'niceguys',
    'onejoke',
    'Feminism',
    'WitchesVsPatriarchy',
    'AreTheStraightsOK',
    'ToiletPaperUSA',
    'LeopardsAteMyFace',
    'TwoXChromosomes',
    'Anarchism',
    'TopMindsOfReddit',
    'esist',
    'MarchAgainstNazis',
    'exredpill',
    'MensLib',
    'lgbt',
    'AgainstHateSubreddits',
    'traaaaaaannnnnnnnnns',
    'trans',
    'GenderCynical',
    'EnoughTrumpSpam',

    #right (I think)
    'dankmemes',
    'PolitcalCompassMemes',
    'antifeminists',
    'MensRights',
    'ProudMaleFeminists',
    'WhereAreTheFeminists',
    'FeminismUncensored',
    'SRSsucks',
    'Conservative',
    'JordanPeterson',
    'conspiracy',
    'tucker_carlson',
    'TheTrumpZone',
    'LouderWithCrowder',
    'Libertarian',
    'AskThe_Donald',
    'walkaway',
    'ConservativesOnly',
    'IncelsCircleJerk',
    'BlackPillScience',
    'Blubber_Whaling',
    'memegender',
    'SocialJusticeInAction',
    'FreeSpeech',
    'libsofreddit',
    'EnoughCommieSpam'


]

In [9]:
# Now, lets build a dataframe of the top posts of all the subreddits in this list

messy_df = pd.DataFrame()
params = {'limit' : 100} # get 100 posts

for subreddit in subreddits:
    print(subreddit)
    response = requests.get(f"https://oauth.reddit.com/r/{subreddit}/top/?t=all", headers=headers, params = params)

    #TODO: There is a lot of data that isn't being captured here that could be added
    for post in response.json()['data']['children']:
        messy_df = messy_df.append({
            'created_utc' : post['data']['created_utc'],
            'subreddit' : post['data']['subreddit'],
            'subreddit_subscribers' : post['data']['subreddit_subscribers'],
            'title' : post['data']['title'],
            'text' : post['data']['selftext'],
            'upvote_ratio' : post['data']['upvote_ratio'],
            'num_comments' : post['data']['num_comments'],
            'url' : post['data']['url'] # this is just a url link to picture media
        }, ignore_index = True)

BlackPeopleTwitter
WhitePeopleTwitter
politics
news
Jokes
dankmemes
antiwork
funny
gaming
technology
dataisbeautiful
MurderedByWords
mildlyinteresting
wallstreetbets
shitposting
pcmasterrace
niceguys
teenagers
PublicFreakout
SelfAwarewolves
dndmemes
memes
meirl
PolitcalCompassMemes
AskReddit
onejoke
Feminism
WitchesVsPatriarchy
AreTheStraightsOK
LeopardsAteMyFace
antifeminists
MensRights
ProudMaleFeminists
TwoXChromosomes
WhereAreTheFeminists
FeminismUncensored
ShitRedditSays
SRSsucks
Conservative
JordanPeterson
Anarchism
TopMindsOfReddit
conspiracy
tucker_carlson
TheTrumpZone
ToiletPaperUSA
LouderWithCrowder
Libertarian
AskThe_Donald
walkaway
esist
ConservativesOnly
MarchAgainstNazis
GamingCircleJerk
IncelsCircleJerk
BlackPillScience
Blubber_Whaling
memegender
SocialJusticeInAction
exredpill
MensLib
lgbt
AgainstHateSubreddits
traaaaaaannnnnnnnnns
trans
GenderCynical
FreeSpeech
libsofreddit


In [10]:
messy_df

,created_utc,subreddit,subreddit_subscribers,title,text,upvote_ratio,num_comments,url
0,1.611842e+09,BlackPeopleTwitter,5549245.0,This is one of the wildest trends in a while imo,,0.95,1082.0,https://i.redd.it/y7fv0axav2e61.jpg
1,1.584018e+09,BlackPeopleTwitter,5549245.0,Gonna come back to a world in shambles,,0.96,1487.0,https://i.redd.it/c4wtpybkp8m41.jpg
2,1.611880e+09,BlackPeopleTwitter,5549245.0,"A fraudulent system, would you agree?",,0.90,1122.0,https://i.imgur.com/BJ67z6S.jpg
3,1.590760e+09,BlackPeopleTwitter,5549245.0,2020 is coming with it all.,,0.91,1152.0,https://i.redd.it/3j1yilhqkp151.jpg
4,1.607440e+09,BlackPeopleTwitter,5549245.0,"One thing about the French, they gon riot",,0.86,2420.0,https://i.redd.it/3s2qe6q3cz361.jpg
...,...,...,...,...,...,...,...,...
6610,1.643646e+09,libsofreddit,17881.0,These triggered libs can't stop posting L's,,0.98,17.0,https://i.redd.it/vqxp0a73s1f81.jpg
6611,1.651523e+09,libsofreddit,17881.0,Someone was being mean to me in the comment se...,,0.97,12.0,https://i.redd.it/7wsb4g1jf4x81.jpg
6612,1.648672e+09,libsofreddit,17881.0,Leftist tiktoker gets kicked out of the house ...,,0.95,123.0,https://v.redd.it/4wm56sihzkq81
6613,1.650651e+09,libsofreddit,17881.0,I guess I survived?,,0.98,19.0,https://i.redd.it/khj9vwhpf4v81.gif
